In [34]:

import tensorflow as tf
import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Check for TensorFlow GPU access
print(tf.config.list_physical_devices())

# See TensorFlow version
print(tf.__version__)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.10.0


In [ ]:
base_dir = '../data/fruits-360/'
test_dir = os.path.join(base_dir, 'Test')
train_dir = os.path.join(base_dir, 'Training')

train_dir = pathlib.Path(train_dir)
test_dir = pathlib.Path(test_dir) 

In [ ]:
# Img size

img_height = 224
img_width = 224

In [31]:
# ResNet50 Model 

# Data preprocessing - ImageDataGenerator

# Normalize the pixels in the train data images, resize and augment the data
train_datagen = ImageDataGenerator(
    rescale=1./255,# Image augmentaion 
    shear_range=0.2,
    zoom_range=0.2, # Zoom in on image by 20%
    horizontal_flip=True, #  Flip the image horizontally
    validation_split=0.2) # Split 20% of the data for validation

# Normalize the test data images
test_datagen = ImageDataGenerator(rescale=1./255) 

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='training') # Set as training data

validation_generator = train_datagen.flow_from_directory(
    train_dir, # Same directory as training data
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation') # Set as validation data

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical')

Found 54190 images belonging to 131 classes.
Found 13502 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [ ]:
# Create the model based on ResNet50

# Load the ResNet50 model
conv_base = keras.applications.ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_shape=(img_height, img_width, 3),
)

pool = GlobalAveragePooling2D()(conv_base.output)
dense = layers.Dense(512, activation = 'relu')(pool)
out_class = layers.Dense(131, activation = 'softmax')(dense)
model_resnet = models.Model(inputs = conv_base.input, outputs=out_class)

# Compile the model
model_resnet.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Model summary
model_resnet.summary()

In [ ]:
# Check for GPU availability
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
# Train the model
with tf.device("/device:GPU:0"):
        history = model_resnet.fit(
        train_generator,
        epochs=5,
        validation_data=validation_generator,
        verbose = 1,
        callbacks=[EarlyStopping(monitor='val_accuracy', patience = 3, restore_best_weights = False)]
        )
        
# Save the model
def save_model(model, seed):
    model.save(f"../models/resnet50v2-{seed}")
    
save_model(model_resnet, 1)

In [ ]:
# Test the model

loss, accuracy = model_resnet.evaluate(test_generator)
print(f'Test accuracy: {accuracy:.3f}, Test loss: {loss:.3f}')

In [21]:
# Predictions

# Load model for predictions
# model_resnet = keras.models.load_model('../models/resnet50v2-1')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

filenames = test_generator.filenames
nb_samples = len(filenames)

predict = model_resnet.predict(test_generator, steps = nb_samples)

Found 22688 images belonging to 131 classes.
22688/22688 [==============================] - 305s 13ms/step


In [29]:
# Get the predicted classes

files = test_generator.filenames
class_dict = test_generator.class_indices # a dictionary of the form class name: class index
rev_dict = {}
for key, value in class_dict.items():
    rev_dict[value] = key   # dictionary of the form class index: class name
    
for i, p in enumerate(predict):
    index = np.argmax(p)
    klass = rev_dict[index]    
    prob = p[index]    
    print(f'Predicted class: {rev_dict[i]}, Probability: {prob:.3f}')

Predicted class: Apple Braeburn, Probability: 0.512
Predicted class: Apple Crimson Snow, Probability: 0.524
Predicted class: Apple Golden 1, Probability: 0.470
Predicted class: Apple Golden 2, Probability: 0.484
Predicted class: Apple Golden 3, Probability: 0.509
Predicted class: Apple Granny Smith, Probability: 0.562
Predicted class: Apple Pink Lady, Probability: 0.287
Predicted class: Apple Red 1, Probability: 0.427
Predicted class: Apple Red 2, Probability: 0.431
Predicted class: Apple Red 3, Probability: 0.531
Predicted class: Apple Red Delicious, Probability: 0.541
Predicted class: Apple Red Yellow 1, Probability: 0.466
Predicted class: Apple Red Yellow 2, Probability: 0.519
Predicted class: Apricot, Probability: 0.595
Predicted class: Avocado, Probability: 0.532
Predicted class: Avocado ripe, Probability: 0.416
Predicted class: Banana, Probability: 0.397
Predicted class: Banana Lady Finger, Probability: 0.392
Predicted class: Banana Red, Probability: 0.581
Predicted class: Beetro

KeyError: 131

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(1, len(history.history['accuracy']) + 1)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.title('Training and Validation Loss')